# Advanced Lane Finding


This project use the following topics to find the lane lines on the road.

* Compensate for Camera Lens Distorion
    - Compute the camera calibration matrix and distortion coefficients.
    - Apply a distortion correction to raw images.
* Create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

The images for camera calibration are stored in the folder called `camera_cal`.  The images in `test_images` are for testing your pipeline on single frames.



## Comensate for Camera Lens Distorion

To be able to correctly get the radius of curvature and massure position inside the lane, we need to calibrate the camera to compansate for lens distorion. 


### Compute the camera calibration matrix and distortion coefficents

computing camera calibration is done in the following steps:
* prepare object point matrix
* convert image to grayscale
* finding chessboard corners
* append detected corners for each image
* [optional] draw corners on the image, for checking for detection
* calibrate camera with detected corners from all images

All these above steps is done in function `calibrate` in `camera.py`, explained in the code bellow, can also be called from `jupyter notebook` or other python scripts as follows:
```python
    import camera
    
    # do calibration on chessboard images
    ret, mtx, dist, rvecs, tvecs = camera.calibrate('camera_cal', 9, 6)
    
    # store all data from calibration
    camera.store('camera_calibration_data.p', ret, mtx, dist, rvecs, tvecs)
```

Calibration can also be executed standalone with `camera.py`, from console as this example:
```shell
$ python camera.py --ipath camera_cal --numx 9 --numy 6 -o camera_calibration_data.p
```  

It's also possible to store the results from corner detection, for sanity check purposes, for more info on that use parameter `-h` or `--help` to get more information.

#### prepating object point matrix

Operation for creating object point matrix:
```python
    objpnt = np.zeros((ny*nx, 3), np.float32)
    objpnt[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
```
Here `nx` and `ny` correspons number of corners in x and y direction on the chessboard calibration images, given as paramter `--numx` and `--numy` in the command line tool. Look at the number of detected points in x and y direction in the `example of corner finding result` image bellow.

#### convert image to grayscale

This is done by `cv2.cvtColor` function as follows:
```python
    # convert image to grayscale 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
```


#### finding chessboard corners

The process of finding chessboard corners is done as follows:
```python
    #find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
```
where `nx` and `ny` are number of intersecting corners in `x` and `y` direction, which is 9 and 6 for the example images bellow. 

<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>example of corner finding result</H3></center></td>
  </tr>
  <tr>
    <td><img src="./output_images/corners_calibration2.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/corners_calibration3.jpg" width="90%" height="90%"/></td>
  </tr>
  <tr>
    <td><center>calibration2</center></td>
    <td><center>calibration3</center></td>
  </tr>
</table> 


#### append detected corners

The detected corners is then added to imgpoints array, for being used later for calibration of camera.

```python
    # append detected corners 
    imgpoints.append(corners)
    objpoints.append(objpnt)
```

#### calibrate camera with detected corners from all images

When above corner detection and collection of results have been done for all chessboards images, then it is time to calculate the distortion matrix `mtx` and the distortion coefficents `dist` as shown bellow. 

```python
    # gives the ret, mtx, dist, rvecs, tvecs
    cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
```

The result `mtx` and `dist` can be stored for later use, so that this process does not need to be recalculated every time. Only needes to be recalculate if the camera needs to be changed.


### Apply a distortion correction to raw images


<table style="width:100%">
  <tr>
      <td colspan="3"><center><H3>camera distortion correction</H3></center></td>
  </tr>
  <tr>
    <td><img src="./test_images/test1.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/undist_diff_test1.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/undist_test1.jpg" width="90%" height="90%"/></td>
  </tr>
  <tr>
    <td><center>original</center></td>
    <td><center>diff between images</center></td>
    <td><center>undistorted</center></td>
  </tr>
</table> 

The images above shows the original camera image to the left, undistored image to the right, and the difference between the two images is shown in the center. Notes that the there is no difference in the center of the image, which indicates that there is no distortion there.


Loading the camera calibration matrix and distortion coeffcients from file, is done as follows.

```python
    import camera
    
    # load all data from calibration file
    data = camera.load('camera_calibration_data.p')
    
    # get the mtx and dist from dictonary
    mtx = data["mtx"]
    dist = data["dist"]
```

Then the undistorted image can obtained as follows:
```python
    # obtain the undistored image
    undistorted = cv2.undistort(image, mtx, dist, None, mtx)
```

**Note**: This process is done in the initialization of lane detector class:
```python
    from pipeline import lane_detector
    
    # create a new lane detector
    detect = lane_detector('camera_calibration_data.p')
```

## Create a thresholded binary image


<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>color binary example images</H3></center></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/binary/test1_binary.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/binary/test2_binary.jpg" width="90%" height="90%"/></td>
  </tr>
  <tr>
    <td><center>test1</center></td>
    <td><center>test2</center></td>
  </tr>
</table> 

Above images shows the result of creating a threshold image. 

### applying color transform and gradients

I choose the L from LUV color space with color threshold 200-255 (green color), R channel from RGB with color threshold 225-255 (blue color) and apply a sobel magnitude on B channel from LAB color space with threshold 145-200 (red color). The L in LUV were often issue of overfitting, due to noise furhter away from the car, which can be seen for the `test1` image file (see warped image below).

The bellow code shows howto apply absolute gradient $|\partial x|$ in $x$ direction:
```python
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx) 
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
```
**Note**: it is important to scale the result to the values in $[0-255]$, which is the range of values of a color elememt of one pixel. Also, note that we only work with one color channel.

The magnitude of gradients is taken as $mag = \sqrt{{\partial x}^2 + {\partial y}^2 }$, and scaled back to 8 bit value $[0-255]$ as follows:

```python
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the magnitude 
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
```

Performing threshold operation on is done as follows:
```python
    binary = np.zeros_like(channel)
    binary[(channel >= low) & (channel <= high)] = 1
```
where `channel` is image with one color channel, and where `low` and `high` is the low and high values of the threshold.  


## Apply a perspective transform

to rectify binary image ("birds-eye view")


The perspective transformation is performed in two stages with opencv:
* getPerspectiveTransform
* warpPerspective

The first operation takes in a source (blue) and destination (green) rectangle as parameter, and returns a transformation matrix. The images bellow shows the used source (blue) and destination (green) rectangle, for two of the images.


<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>source and destination lines</H3></center></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/lines/test1_lines.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/lines/test2_lines.jpg" width="90%" height="90%"/></td>
  </tr>
  <tr>
    <td><center>test1</center></td>
    <td><center>test2</center></td>
  </tr>
</table> 




The images bellow shows the result after second operation have been applied. The left side images bellow represent the left image above, and the right side images bellow represent the right side image above. The top image bellow on each side is the actual warped image, while the bottom image bellow shows which elements is contributing to the warped result.

<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>warped binary and color binary images</H3></center></td>
  </tr>
  <tr>
    <td><center>test1</center></td>
    <td><center>test2</center></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/warped/test1_warped.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/warped/test2_warped.jpg" width="90%" height="90%"/></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/color/test1_warped.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/color/test2_warped.jpg" width="90%" height="90%"/></td>
  </tr> 
   
</table>


What can be observed from these images is that the blue component, sobel magnitude of L channel, is more present than we like it to be. The left image is the same image is shown in the distorion correction example,above subsection section marked with 2. So, this is a image with light road pavement, which I have not been able to find sutable adjustment for with help  of gradients and threshold values. 

The left image `test1` have alot of noise which is due to the L channel in LUV color space, see the green color in `test1` image. Need an adaptive way to change the threshold value of L channel, based brightnes of the color relative to distance.

## Detect lane pixels and fit to find the lane boundary


The detection of lane lines starts with creating a histogram of all non zero pixels in y-direction (image height), for every pixel in x-direction (image width). With this histogram one can find peak values for left and right side of center of the image width, which will give the lane lines closesed to the car (if step 4 have given good results). 


<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>lower half of binary togher with histogram</H3></center></td>
  </tr>
  <tr>
    <td><center>test1</center></td>
    <td><center>test2</center></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/histogram/test1.png" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/histogram/test2.png" width="90%" height="90%"/></td>
  </tr>
</table> 


When these left and right peaks have been found, the detection algorithm is as follows:
* devide detection in parts from bottom up 
    - define a rectagle as can be seen bellow
    - start at the lowest part of the image
* identify all none zero pixels inside the rectangle
* take the mean of all x values of none zero pixel the rectagle
    - to be able to center the next rectagle when moving upwards to top of the image
* when all 9 iteration rectagles on each side have been processed
    - then we have a collection of pixels inside these rectagles shown bellow
    - these pixels will then be used to fit a polynomial curve

<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>lower half of binary togher with histogram</H3></center></td>
  </tr>
  <tr>
    <td><center>test1</center></td>
    <td><center>test2</center></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/output/test1_output.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/output/test2_output.jpg" width="90%" height="90%"/></td>
  </tr>
</table> 

The `np.polyfit` function will produce a second degree polynomial in the form (giving by $a$, $b$ and $c$:
$$
    x = f(x) = ay^2 + by + c
$$
by giving use $a$, $b$ and $c$


## Radius of curvature and vehicle position

The [radius of curvature](https://www.intmath.com/applications-differentiation/8-radius-curvature.php) is calculated from the following formula:
$$
    \rho  = \frac{\left[ 1 + {f^{'}}^2(y)\right]^{{}^3/_2} }{\left| f^{''}(y)\right|}
$$

where $f(y)$ is a polynomial curve obtained from previous step, defined as:
$$
    x = f(x) = ay^2 + by + c
$$

The 1st and 2nd derivative of $f(x)$ is easily obtained as:

$$
    f^{'}(y) = 2ay + b\,\,\,\, and \,\,\,\, f^{''}(y) = 2a
$$

Then the left and right radius of curvature is obtained as follows:
```python
    # adjust the y in [0..h] by meter per pixel 
    y = y_eval * ym_per_pix
    
    a = left_fit[0]
    b = left_fit[1]
    # calculating left radius of curvature 
    left_curverad = ((1 + (2*a*y + b)**2)**1.5) / np.absolute(2*a)
    
    a = right_fit[0]
    b = right_fit[1]
    # calculating right radius of curvature
    right_curverad = ((1 + (2*a*y + b)**2)**1.5) / np.absolute(2*a)
```

The vehicle position in the lane is calculated by first taking the diff of left and right detected peak, see histogram images above. This is done as follows:

```python
    left = left_peak - midpoint
    right = right_peak - midpoint
    diff = right + left
```    
The sign of the diff is used to indicate if car is left or right side of the center of lane.  
  

By using lane size 3.7m and deviding total number of pixels between left and right peak, one can calcluate an approximate distance from lane center by multiplying with absolute diff.
```python
    # vehicle lane 3.7m us highway - https://en.wikipedia.org/wiki/Lane
    lane_size = 3.7
    value = np.abs(diff) * (lane_size / (np.abs(left) + np.abs(right)))
```

## Warp the detected lane boundaries into image space.

The `output` from detection of lane boundaries in warped space must be transformed back into image space, and this is done by first calculating the inverse matrix $M^{-1}$ by changing the `src` and `dst` points when using the `getPerspectiveTransform` function. Then the `output` (warped space image) can be used to produce a unwarped (image space image), which can then be used to blend with the undistored image.

```python
    # only width and height, discard depth if provided
    w,h = binary.shape[1::-1]
        
    # get Minv, the transform matrix
    Minv = cv2.getPerspectiveTransform(dst, src)
        
    # returned the warped image
    unwarped = cv2.warpPerspective(output, Minv, (w,h), flags=cv2.INTER_LINEAR)
    
    # mix undist image and the unwarped 'detection' image
    result = cv2.addWeighted(undist, 0.9, unwarped, 0.2, 0)
```

<table style="width:100%">
  <tr>
      <td colspan="2"><center><H3>unwarp detection result and blending with undistorted image</H3></center></td>
  </tr>
  <tr>
    <td><center>test1</center></td>
    <td><center>test2</center></td>
  </tr>
  <tr>
    <td><img src="./output_images/test_images/result/test1_result.jpg" width="90%" height="90%"/></td>
    <td><img src="./output_images/test_images/result/test2_result.jpg" width="90%" height="90%"/></td>
  </tr>
</table> 